## Сравнение различных стратегий в дилемме заключённого

##### Класс экперимент и определение стратегий

In [7]:
import numpy as np
from typing import Callable, Tuple, List, Dict
from collections import defaultdict
import random

class Experiment:
    
    def __init__(self, strategy_a: Callable, strategy_b: Callable, num_rounds: int = 200):

        self.strategy_a = strategy_a
        self.strategy_b = strategy_b
        self.num_rounds = num_rounds
        
        self.payoff_matrix = {
            (0, 0): (3, 3),  
            (0, 1): (0, 5),  
            (1, 0): (5, 0),  
            (1, 1): (1, 1)   
        }
        
        self.history_a = []  
        self.history_b = []  
        self.scores_a = []   
        self.scores_b = []   
        
    def play_round(self, round_num: int) -> Tuple[int, int]:
        prev_moves_a = self.history_a[:round_num]
        prev_moves_b = self.history_b[:round_num]
        
        move_a = self.strategy_a(prev_moves_a, prev_moves_b, round_num)
        move_b = self.strategy_b(prev_moves_b, prev_moves_a, round_num)
        
        return move_a, move_b
    
    def run_experiment(self) -> Dict:
        self.history_a = []
        self.history_b = []
        self.scores_a = []
        self.scores_b = []
        
        for round_num in range(self.num_rounds):
            move_a, move_b = self.play_round(round_num)
            
            self.history_a.append(move_a)
            self.history_b.append(move_b)
            
            score_a, score_b = self.payoff_matrix[(move_a, move_b)]
            self.scores_a.append(score_a)
            self.scores_b.append(score_b)

        return self.analyze_results()
    
    def analyze_results(self) -> Dict:
        total_score_a = sum(self.scores_a)
        total_score_b = sum(self.scores_b)

        dominant_series_a = self.find_dominant_series(self.scores_a, self.scores_b)
        dominant_series_b = self.find_dominant_series(self.scores_b, self.scores_a)
        
        return {
            'total_scores': (total_score_a, total_score_b),
            'dominant_series': (dominant_series_a, dominant_series_b),
            'moves_a': self.history_a,
            'moves_b': self.history_b,
            'scores_a': self.scores_a,
            'scores_b': self.scores_b
        }
    
    def find_dominant_series(self, scores1: List[int], scores2: List[int]) -> int:
        max_series = 0
        current_series = 0
        
        for s1, s2 in zip(scores1, scores2):
            if s1 == 5 and s2 == 0:
                current_series += 1
                max_series = max(max_series, current_series)
            else:
                current_series = 0
        
        return max_series

def calculate_statistics(samples: List[float]) -> Dict:
    """Вычисляет статистические характеристики выборки"""
    if not samples:
        return {}
    
    n = len(samples)
    
    mean = sum(samples) / n
    
    variance = sum((x - mean) ** 2 for x in samples) / (n - 1) if n > 1 else 0

    sorted_samples = sorted(samples)
    if n % 2 == 1:
        median = sorted_samples[n // 2]
    else:
        median = (sorted_samples[n // 2 - 1] + sorted_samples[n // 2]) / 2

    k = int(np.log2(1 + n))
    if k < 1:
        k = 1
    
    min_val, max_val = min(samples), max(samples)
    bin_width = (max_val - min_val) / k if max_val > min_val else 1
   
    frequencies = [0] * k
    for x in samples:
        if bin_width > 0:
            bin_idx = min(int((x - min_val) / bin_width), k - 1)
        else:
            bin_idx = 0
        frequencies[bin_idx] += 1

    max_freq_idx = np.argmax(frequencies)
    mode_interval_low = min_val + max_freq_idx * bin_width
    mode_interval_high = mode_interval_low + bin_width

    mode = (mode_interval_low + mode_interval_high) / 2
    
    return {
        'mean': mean,
        'variance': variance,
        'median': median,
        'mode': mode,
        'samples': samples
    }

def run_multiple_simulations(strategy1, strategy2, num_simulations=1000):
    scores1 = []
    scores2 = []
    
    for _ in range(num_simulations):
        experiment = Experiment(strategy1, strategy2)
        result = experiment.run_experiment()
        scores1.append(result['total_scores'][0])
        scores2.append(result['total_scores'][1])
    
    stats1 = calculate_statistics(scores1)
    stats2 = calculate_statistics(scores2)
    
    return {
        'strategy1_stats': stats1,
        'strategy2_stats': stats2,
        'all_scores1': scores1,
        'all_scores2': scores2
    }

def strategy_alex(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 1

def strategy_bob(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 0

def strategy_clara(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    return opp_prev[-1] if opp_prev else 0

def strategy_denis(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    return 1 - opp_prev[-1] if opp_prev else 0

def strategy_emma(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 1 if (round_num + 1) % 20 == 0 else 0

def strategy_frida(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if not opp_prev:
        return 0
    return 0 if all(move == 0 for move in opp_prev) else 1

def strategy_merciful_clara(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    
    if (round_num + 1) % 10 == 0:
        return 0

    return opp_prev[-1] if opp_prev else 0

def strategy_hank(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return random.randint(0, 1)

def strategy_ivan(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    return 0 if random.random() < 0.9 else 1

def strategy_jack(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    
    if opp_prev[-1] == 0:
        return 0
    else:
        return 0 if random.random() < 0.25 else 1

def strategy_kevin(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    
    if random.random() < 0.25:
        return 1 - opp_prev[-1]
    else:
        return opp_prev[-1]

def strategy_lucas(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if not hasattr(strategy_lucas, 'period'):
        strategy_lucas.period = random.randint(1, 50)
    
    return 1 if (round_num + 1) % strategy_lucas.period == 0 else 0

def strategy_max(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if not hasattr(strategy_max, 'current_move'):
        strategy_max.current_move = 0
        strategy_max.remaining = random.randint(0, 20)
    
    if strategy_max.remaining == 0:
        strategy_max.current_move = 1 - strategy_max.current_move
        strategy_max.remaining = random.randint(0, 20)
    
    strategy_max.remaining -= 1
    return strategy_max.current_move

def strategy_natan(own_prev: List[int], opp_prev: List[int], round_num: int) -> int:
    if round_num == 0:
        return 0
    
    if len(opp_prev) > 0:
        cooperation_rate = sum(1 for move in opp_prev if move == 0) / len(opp_prev)
    else:
        cooperation_rate = 0.5

    prob_cooperate = 0.3 + 0.4 * cooperation_rate
    
    return 0 if random.random() < prob_cooperate else 1

def calculate_strategy_rankings():
    strategies = {
        'Alex': strategy_alex,
        'Bob': strategy_bob,
        'Clara': strategy_clara,
        'Denis': strategy_denis,
        'Emma': strategy_emma,
        'Frida': strategy_frida,
        'merciful_Clara': strategy_merciful_clara
    }
    
    total_scores = defaultdict(int)
    total_games = defaultdict(int)
    max_series = defaultdict(int)
    avg_scores = {}
    
    strategy_names = list(strategies.keys())

    for i, name1 in enumerate(strategy_names):
        for j, name2 in enumerate(strategy_names):
            experiment = Experiment(strategies[name1], strategies[name2])
            result = experiment.run_experiment()
            
            total_scores[name1] += result['total_scores'][0]
            total_scores[name2] += result['total_scores'][1]
            
            max_series[name1] = max(max_series[name1], result['dominant_series'][0])
            max_series[name2] = max(max_series[name2], result['dominant_series'][1])
            
            total_games[name1] += 1
            total_games[name2] += 1
    
    for strategy in strategy_names:
        avg_scores[strategy] = total_scores[strategy] / total_games[strategy]
    
    sorted_by_avg_score = sorted(avg_scores.items(), key=lambda x: x[1], reverse=True)
    sorted_by_total_score = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)
    sorted_by_max_series = sorted(max_series.items(), key=lambda x: x[1], reverse=True)
    
    return {
        'by_avg_score': sorted_by_avg_score,
        'by_total_score': sorted_by_total_score,
        'by_max_series': sorted_by_max_series,
        'total_scores': dict(total_scores),
        'avg_scores': avg_scores,
        'max_series': dict(max_series)
    }

def compare_all_strategies():
    deterministic_strategies = {
        'Alex': strategy_alex,
        'Bob': strategy_bob,
        'Clara': strategy_clara,
        'Denis': strategy_denis,
        'Emma': strategy_emma,
        'Frida': strategy_frida
    }
    
    stochastic_strategies = {
        'Hank': strategy_hank,
        'Ivan': strategy_ivan,
        'Jack': strategy_jack,
        'Kevin': strategy_kevin,
        'Lucas': strategy_lucas,
        'Max': strategy_max,
        'Natan': strategy_natan
    }
    
    all_strategies = {**deterministic_strategies, **stochastic_strategies}
    results = {}

    for name1, strat1 in all_strategies.items():
        for name2, strat2 in all_strategies.items():
            key = f"{name1} vs {name2}"
    
            if name1 in stochastic_strategies or name2 in stochastic_strategies:
                results[key] = run_multiple_simulations(strat1, strat2, 100)
            else:
                experiment = Experiment(strat1, strat2)
                single_result = experiment.run_experiment()
                results[key] = {
                    'strategy1_stats': calculate_statistics([single_result['total_scores'][0]]),
                    'strategy2_stats': calculate_statistics([single_result['total_scores'][1]]),
                    'all_scores1': [single_result['total_scores'][0]],
                    'all_scores2': [single_result['total_scores'][1]]
                }
    
    return results, all_strategies

def calculate_overall_rankings():
    results, strategies = compare_all_strategies()
    
    total_scores = defaultdict(list)
    strategy_names = list(strategies.keys())
    
    for name1 in strategy_names:
        for name2 in strategy_names:
            key = f"{name1} vs {name2}"
            if key in results:
                total_scores[name1].extend(results[key]['all_scores1'])
                total_scores[name2].extend(results[key]['all_scores2'])

    strategy_stats = {}
    for strategy, scores in total_scores.items():
        strategy_stats[strategy] = calculate_statistics(scores)

    sorted_by_mean = sorted(strategy_stats.items(), 
                          key=lambda x: x[1]['mean'], reverse=True)
    
    return {
        'by_mean': sorted_by_mean,
        'all_stats': strategy_stats,
        'total_scores': total_scores
    }

def print_detailed_comparison():
    rankings = calculate_overall_rankings()
    
    print("=" * 80)
    print("СРАВНЕНИЕ ДЕТЕРМИНИРОВАННЫХ И СТОХАСТИЧЕСКИХ СТРАТЕГИЙ")
    print("=" * 80)
    print("\nРЕЙТИНГ СТРАТЕГИЙ ПО СРЕДНЕМУ КОЛИЧЕСТВУ ОЧКОВ:")
    print("-" * 80)
    print(f"{'Стратегия':<10} {'Среднее':<8} {'Медиана':<8} {'Мода':<8} {'Дисперсия':<10} {'Тип':<12}")
    print("-" * 80)
    
    deterministic = ['Alex', 'Bob', 'Clara', 'Denis', 'Emma', 'Frida']
    
    for strategy, stats in rankings['by_mean']:
        strategy_type = "Детерминир." if strategy in deterministic else "Стохастич."
        print(f"{strategy:<10} {stats['mean']:<8.1f} {stats['median']:<8.1f} "
              f"{stats['mode']:<8.1f} {stats['variance']:<10.1f} {strategy_type:<12}")
    
    print("\n" + "=" * 50)
    print("ТОП-5 СТРАТЕГИЙ:")
    print("=" * 50)
    for i, (strategy, stats) in enumerate(rankings['by_mean'][:5], 1):
        print(f"{i}. {strategy}: Среднее = {stats['mean']:.1f}, "
              f"Медиана = {stats['median']:.1f}, Дисперсия = {stats['variance']:.1f}")

def analyze_stochastic_vs_deterministic():
    rankings = calculate_overall_rankings()
    
    deterministic_scores = []
    stochastic_scores = []
    
    deterministic = ['Alex', 'Bob', 'Clara', 'Denis', 'Emma', 'Frida']
    
    for strategy, stats in rankings['all_stats'].items():
        if strategy in deterministic:
            deterministic_scores.append(stats['mean'])
        else:
            stochastic_scores.append(stats['mean'])
    
    if deterministic_scores and stochastic_scores:
        avg_det = sum(deterministic_scores) / len(deterministic_scores)
        avg_stoch = sum(stochastic_scores) / len(stochastic_scores)
        
        print("\n" + "=" * 60)
        print("СРАВНЕНИЕ ДЕТЕРМИНИРОВАННЫХ И СТОХАСТИЧЕСКИХ СТРАТЕГИЙ:")
        print("=" * 60)
        print(f"Средний результат детерминированных стратегий: {avg_det:.2f}")
        print(f"Средний результат стохастических стратегий: {avg_stoch:.2f}")
        print(f"Разница: {abs(avg_det - avg_stoch):.2f}")
        



##### Сравнение стратегий

In [8]:
if __name__ == "__main__":
    print_detailed_comparison()
    analyze_stochastic_vs_deterministic()

СРАВНЕНИЕ ДЕТЕРМИНИРОВАННЫХ И СТОХАСТИЧЕСКИХ СТРАТЕГИЙ

РЕЙТИНГ СТРАТЕГИЙ ПО СРЕДНЕМУ КОЛИЧЕСТВУ ОЧКОВ:
--------------------------------------------------------------------------------
Стратегия  Среднее  Медиана  Мода     Дисперсия  Тип         
--------------------------------------------------------------------------------
Frida      581.4    599.0    870.5    56332.5    Детерминир. 
Max        572.7    440.0    212.4    106366.4   Стохастич.  
Alex       555.8    448.0    400.0    64743.6    Детерминир. 
Denis      517.9    478.0    449.1    77399.2    Детерминир. 
Clara      473.8    474.5    580.0    16586.9    Детерминир. 
Hank       470.9    463.0    457.5    57413.2    Стохастич.  
Jack       470.5    564.0    578.5    24545.6    Стохастич.  
Natan      461.2    468.0    436.5    40436.6    Стохастич.  
Kevin      458.5    465.0    434.0    36379.8    Стохастич.  
Emma       416.1    458.0    589.5    36444.3    Детерминир. 
Bob        406.0    453.0    570.0    36397.8    Дет